In [28]:
# Import necessary libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
import dash
from dash import Dash, dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
from crud import AnimalShelter
import matplotlib.pyplot as plt

# Setup the animal shelter connection
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter()

# Read data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Drop '_id' column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Ensure that the DataFrame has the expected columns for mapping
expected_columns = [
    "name", "animal_type", "breed", "location_lat", "location_long",
    "rec_num", "age_upon_outcome", "animal_id", "color", "date_of_birth",
    "datetime", "monthyear", "outcome_subtype", "outcome_type",
    "sex_upon_outcome", "age_upon_outcome_in_weeks"
]
for col in expected_columns:
    if col not in df.columns:
        df[col] = None

# Initialize the Dash app
app = JupyterDash('SimpleExample')

# Layout of the app
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='buttonRow',
            style={'display' : 'flex'},
            children=[
                html.Button(id='submit-button-one', n_clicks=1, children='Cats'), 
                html.Button(id='submit-button-two', n_clicks=1, children='Dogs')
            ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],  # Select the first row by default
        page_size=10,  # Display 10 rows per page
        style_table={'height': '300px', 'overflowY': 'auto'},
        style_cell={
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    )
])

# Callback to update row selection style
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

@app.callback(Output('datatable-id', "data"),
             [Input('submit-button-one', 'n_clicks'),
             Input('submit-button-two', 'n_clicks')
              ])
def on_click(button1, button2):
    df =pd.DataFrame.from_records(shelter.read({}))
    if(int(button1) > int(button2)):
        df= pd.DataFrame.from_records(shelter.read({"animal_type" : "Cat"}))
    elif (int(button2) > int(button1)):
        df= pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog"}))
    
    #Cleanup Mongo_id field
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

def update_styles(selected_columns):
    if selected_columns is None:
        selected_columns = []
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]

# Callback to update map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)


def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available")]

    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if index is None or empty
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Ensure the index is within bounds
    if row >= len(dff):
        row = 0
    
    # Handle missing or invalid location data
    try:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        if pd.isnull(lat) or pd.isnull(lon):
            lat, lon = 30.75, -97.48  # Default to Austin, TX
    except (KeyError, IndexError):
        lat, lon = 30.75, -97.48  # Default to Austin, TX

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon],
                      children=[
                          dl.Tooltip(dff.iloc[row]['breed']),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row]['name'])
                          ])
                      ])
        ])
    ]

# Run the app
app.run_server(debug=True)


Connected to MongoDB database: AAC, collection: animals
Query result: [{'_id': ObjectId('66702ce67c862c85227d29af'), 'rec_num': 4, 'age_upon_outcome': '2 years', 'animal_id': 'A716331', 'animal_type': 'Dog', 'breed': 'Pitbull', 'color': 'Brown/White', 'date_of_birth': '2013-11-19', 'datetime': '2015-12-28 18:47:00', 'monthyear': '2015-12-28T18:43:00', 'name': 'Broski', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.7595748121648, 'location_long': -97.5523753807133, 'age_upon_outcome_in_weeks': 120}, {'_id': ObjectId('66702d237c862c85227d29b1'), 'rec_num': 5, 'age_upon_outcome': '2 years', 'animal_id': 'A716441', 'animal_type': 'Cat', 'breed': 'Pitbull', 'color': 'Brown/White', 'date_of_birth': '2013-11-19', 'datetime': '2015-12-28 18:47:00', 'monthyear': '2015-12-28T18:43:00', 'name': 'Ulisy', 'outcome_subtype': 'Successfully placed', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Female', 'location_lat': 60.7595748